In [1]:
import pandas as pd

# ---------------------------------------------- #

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# ---------------------------------------------- #

from tensorflow import lite
from IPython.display import display

In [2]:
data = pd.read_csv("../input/dcs-model-deployment-workshop/preprocessed.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   income                   480 non-null    float64
 1   coincome                 480 non-null    float64
 2   amount                   480 non-null    float64
 3   term                     480 non-null    float64
 4   credit_history           480 non-null    int64  
 5   property_area_Semiurban  480 non-null    int64  
 6   property_area_Urban      480 non-null    int64  
 7   gender_Male              480 non-null    int64  
 8   married_Yes              480 non-null    int64  
 9   dependents_1             480 non-null    int64  
 10  dependents_2             480 non-null    int64  
 11  dependents_3+            480 non-null    int64  
 12  education_Not Graduate   480 non-null    int64  
 13  self_employed_Yes        480 non-null    int64  
 14  loan_status_Y            4

In [4]:
# Defining the model
model = Sequential([
    Dense(16, input_shape = (14, )),    # 14 inputs
    Dense(8),
    Dense(1, activation = "sigmoid")
])

# Compile
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

2022-05-22 03:32:07.220670: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
# Training the network
model.fit(data.iloc[:, :-1], data.iloc[:, -1], epochs = 25, batch_size = 16)

2022-05-22 03:32:07.414823: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25
30/30 [==============================] - 1s 1ms/step - loss: 0.6286 - accuracy: 0.6771
Epoch 2/25
30/30 [==============================] - 0s 1ms/step - loss: 0.5889 - accuracy: 0.6875
Epoch 3/25
30/30 [==============================] - 0s 1ms/step - loss: 0.5698 - accuracy: 0.7229
Epoch 4/25
30/30 [==============================] - 0s 1ms/step - loss: 0.5611 - accuracy: 0.7375
Epoch 5/25
30/30 [==============================] - 0s 1ms/step - loss: 0.5494 - accuracy: 0.7208
Epoch 6/25
30/30 [==============================] - 0s 1ms/step - loss: 0.5400 - accuracy: 0.7458
Epoch 7/25
30/30 [==============================] - 0s 1ms/step - loss: 0.5320 - accuracy: 0.7458
Epoch 8/25
30/30 [==============================] - 0s 1ms/step - loss: 0.5283 - accuracy: 0.7500
Epoch 9/25
30/30 [==============================] - 0s 1ms/step - loss: 0.5232 - accuracy: 0.7521
Epoch 10/25
30/30 [==============================] - 0s 1ms/step - loss: 0.5156 - accuracy: 0.7646
Epoch 11/25
30/30 [

In [6]:
# Saving the model
model.save("./model")

2022-05-22 03:32:09.520838: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [7]:
# Inference
from tensorflow.keras.models import load_model
import numpy as np

m = load_model("./model")
m.predict(np.array(data.iloc[1, :-1]).reshape(1, 14))

array([[0.7629219]], dtype=float32)

# **Model quantization using TFLite**

In [8]:
# Initialising converters
tflite_converter = lite.TFLiteConverter.from_saved_model("./model")

# Defining quantisation schema
tflite_converter.optimizations = [lite.Optimize.DEFAULT]

tflite_converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

# Converting to TFLite
tflite_model = tflite_converter.convert()

2022-05-22 03:32:10.746310: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2022-05-22 03:32:10.746378: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:354] Ignored drop_control_dependency.
2022-05-22 03:32:10.746400: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored change_concat_input_ranges.
2022-05-22 03:32:10.748228: I tensorflow/cc/saved_model/reader.cc:38] Reading SavedModel from: ./model
2022-05-22 03:32:10.751150: I tensorflow/cc/saved_model/reader.cc:90] Reading meta graph with tags { serve }
2022-05-22 03:32:10.751185: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: ./model
2022-05-22 03:32:10.751331: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-05-22 03:32:10.758027: I tensorflow/cc/saved_model/loader

In [9]:
# Saving quantised model
with open("./model.tflite", 'wb') as f:
    f.write(tflite_model)